#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [2]:
import os, sys

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [3]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [ ]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1*365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [5]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())
if repo_root == os.getcwd(): #then go one directory above to reach repo root
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [33]:

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["OPTIEMUS.NS","TBZ.NS","DCBBANK.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-10-18 10:05:47,451 - INFO - 📈 Fetching data for 2178 tickers...
2025-10-18 10:05:47,454 - INFO - Fetching data from 2023-09-19 to 2025-10-18


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*********************100%***********************]  2178 of 2178 completed
2025-10-18 10:09:22,723 - ERROR - 
1 Failed download:
2025-10-18 10:09:22,723 - ERROR - ['NDTV-RE.NS']: YFInvalidPeriodError("NDTV-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
2025-10-18 10:09:31,001 - INFO - ✅ Download completed in 223.51 seconds
2025-10-18 10:09:41,386 - INFO - 📊 Final dataset: 1,028,467 rows from 2023-09-20 to 2025-10-17


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2023-09-20,20MICRONS.NS,NaN,123.091866,126.407970,119.033350,119.033350,33118.0,123.091866,123.091866,123.091866,123.091866,NaN,NaN,20MICRONS
1,2023-09-21,20MICRONS.NS,NaN,124.428200,125.220109,121.953494,123.735285,41392.0,123.219135,123.144271,123.118328,123.105162,NaN,NaN,20MICRONS
2,2023-09-22,20MICRONS.NS,NaN,127.447334,130.070516,123.735276,125.220099,82161.0,123.621821,123.313018,123.204051,123.148368,NaN,NaN,20MICRONS
3,2023-09-25,20MICRONS.NS,NaN,131.258408,132.644237,127.694825,128.684707,55088.0,124.349115,123.624602,123.363543,123.229065,NaN,NaN,20MICRONS
4,2023-09-26,20MICRONS.NS,NaN,128.833176,133.436137,128.288752,133.436137,20218.0,124.776168,123.828860,123.471852,123.284827,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [ ]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,INFIBEAM,2025-10-14,2.197781e+06,85.0,1.63,68.04,True,17.490480,16.878134,19.053079,18.639999,3.047770e+07,1.384773e+07
1,MRPL,2025-10-14,6.503044e+05,90.0,1.68,65.26,True,137.770038,134.208113,139.141708,141.009995,5.727355e+06,3.516882e+06
2,TICL,2025-10-14,3.582739e+05,50.0,1.78,74.29,False,26.877594,26.551229,31.733777,30.459999,1.380451e+06,7.439027e+05
3,ORCHASP,2025-10-14,2.976880e+05,55.0,1.65,88.30,False,3.024615,2.899069,3.049188,3.890000,1.392519e+06,6.104664e+05
4,SKYGOLD,2025-10-14,2.823914e+05,70.0,1.51,63.46,False,302.231587,298.215153,314.914336,324.549988,2.125439e+06,9.534426e+05


#### EMA Signal indicator - TBC

##### Update the Stock returns table

In [35]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 2047
✅ Total signals after merging: 308259
📦 Existing SIGNAL_RETURNS records: 0
✅ New unique records found: 2026
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
0,20MICRONS,Golden cross,2025-07-17,254.0,6.43,270.329987,2025-07-24,-7.69,234.470001,2025-07-31,-10.47,227.399994,2025-08-18,-14.98,215.960007,2025-10-15,NaN,NaN,NaT,NaN,NaN,NaT,-15.52,214.570007,2025-10-17,6.43,-7.69,-10.47,-14.98,<NA>,<NA>,-15.52
1,20MICRONS,Golden cross,2025-07-16,259.0,6.86,276.779999,2025-07-23,-5.46,244.850006,2025-07-30,-12.20,227.399994,2025-08-18,-17.18,214.500000,2025-10-14,NaN,NaN,NaT,NaN,NaN,NaT,-17.15,214.570007,2025-10-17,6.86,-5.46,-12.2,-17.18,<NA>,<NA>,-17.15
2,20MICRONS,Golden cross,2025-06-17,237.0,-4.12,227.243713,2025-06-24,-2.58,230.897217,2025-07-01,7.12,253.863495,2025-07-17,-4.95,225.270004,2025-09-15,NaN,NaN,NaT,NaN,NaN,NaT,-9.46,214.570007,2025-10-17,-4.12,-2.58,7.12,-4.95,<NA>,<NA>,-9.46
3,360ONE,Golden cross,2025-06-12,1081.0,3.09,1114.400024,2025-06-19,7.76,1164.849976,2025-06-26,9.92,1188.199951,2025-07-14,-0.28,1078.000000,2025-09-10,NaN,NaN,NaT,NaN,NaN,NaT,7.76,1164.900024,2025-10-17,3.09,7.76,9.92,-0.28,<NA>,<NA>,7.76
4,3MINDIA,Accumulation Signal,2025-08-07,32000.0,-5.16,30350.000000,2025-08-14,-3.52,30875.000000,2025-08-21,-3.64,30835.000000,2025-09-08,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-7.61,29565.000000,2025-10-17,-5.16,-3.52,-3.64,<NA>,<NA>,<NA>,-7.61
